In [ ]:
import datetime
import pandas as pd
import numpy as np
import plotly.express as px


In [ ]:
day = 24 * 60 * 60
year = 365.2425 * day

In [ ]:
firstHarmonic = 5
secondHarmonic = 10
thirdHarmonic = 0
fourthHarmonic = 0
fifthHarmonic = 0

In [ ]:

def load_dataframe(harmonics,sDate,eDate) -> pd.DataFrame:
    """ Create a time series x sin wave dataframe. """
    df = pd.DataFrame(columns=['date', 'sin','combined'])
    df.date = pd.date_range(start=sDate, end=eDate, freq='s')
    df.sin = 1 + np.sin(df.date.astype('int64') // 1e9 * (2 * np.pi / (year/4)))
    df.sin = (df.sin * 100).round(2)
    df.combined = df.sin
    if len(harmonics) > 0:
        for i,h in enumerate(harmonics): 
            col = "h"+ str(i+1)
            df[col] = 1 + np.sin(df.date.astype('int64') // 1e9 * (h * np.pi / day))
            df[col] = (df[col] * 100).round(2)
            df.combined =  df.combined + df[col]

        df.date = df.date.apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
    return df



hs= [0.5,10,2]

In [ ]:
#Input parameters
apiKey = "8332eccddd92a3b6cc005e4dbd288a75"

import ion_sdk.edison_api.edison_api as eapi
from ion_sdk.edison_api.models.factoryModel import EdisonGenericComponent, Model,CurrentState
import time

In [ ]:
## Connect the client
AMAZON_FACTORY_API= 'https://amazon.altergo.io/'
AMAZON_IOT_API= 'https://iot.amazon.altergo.io/'


edApi = eapi.Client(apiKey, AMAZON_FACTORY_API, AMAZON_IOT_API)

In [ ]:
assetSn = "Virtual Site D"
assets = edApi.getAssets(assetSn,200)
asset = None
for a in assets:
    if a.serial_number == assetSn:
        asset = a

In [ ]:
packetTimeLength = 60 # in seconds
dt_s = 60 # time interval between each send.
prev_dt_s = 60
generationIntervalDays = 1

startDate = pd.Timestamp.now(tz='UTC')
endDate = startDate + pd.Timedelta(days=generationIntervalDays*7)

t = startDate
currentDataDf = None

while t < endDate:
    
    if t == startDate:
        print('testStart')
        currentDataDf = load_dataframe(hs,t,t+pd.Timedelta(days=generationIntervalDays))
        lastSend = t 
    
    if t - lastSend >= pd.Timedelta(days=generationIntervalDays):
        print('newGen')
        currentDataDf = load_dataframe(hs,t,t+pd.Timedelta(days=generationIntervalDays))
        lastSend = t 
    
    prev_dt_s = t
    t += pd.Timedelta(seconds=packetTimeLength)
    
    packet = currentDataDf.head(packetTimeLength)
    packet.rename(columns = {'combined':'Generic load'}, inplace = True)
    
    packet['date'] = pd.to_datetime(packet['date'])
    packet = packet.set_index('date')
    
    data = packet
    data = data[['Generic load']].copy()

    uploadSensorList=edApi.refactorDataframeToAsset(data,asset)
    asset.df=data
    edApi.updateSensorDataByFile(asset,uploadSensorList)
    currentDataDf = currentDataDf.iloc[packetTimeLength:]
 
